In [1]:
import glob
import os

In [2]:
xlxs_path = '/Users/soiji/Library/CloudStorage/GoogleDrive-sooh0601@gmail.com/내 드라이브/Data Study/Data Preprocessing Practice/Data'
xlxs_list = os.listdir('Data')

In [3]:
import pandas as pd
import numpy as np

### Problem 1
Re-arrange data into the correct four columns
Columns would be = Segment, Ship Mode, OrderID, Sales
Delete Grand Totals and Sub Totals Data

In [4]:
data = pd.read_excel(xlxs_path + '/' + str(xlxs_list[0]), engine='openpyxl')

전략을 구성해보자.
테이블이 아예 피봇된 형태인거 아님?
index = OrderID
colum = Segment, Shipmode
Value = Cost

In [5]:
# 필요없는 총합 데이터부터 제거합시다
drop_col = [c for c in data.columns if 'Total' in c]
data.drop(drop_col, axis=1, inplace=True)
data.drop(824, axis=0, inplace=True)

In [6]:
# Segment에 따라 Table 분류합시다
df_consumer = data.iloc[:, [0, 1, 2, 3, 4]].copy()
df_corporate = data.iloc[:, [0, 5, 6, 7, 8]].copy()
df_homeoffice = data.iloc[:, [0, 9, 10, 11, 12]].copy()

# LOOP 변수 정의
df_list = [df_consumer, df_corporate, df_homeoffice]
df_name = ['df_consumer', 'df_corporate', 'df_homeoffice']

final_frame = pd.DataFrame([])

# Table Name 지정
for i, d in enumerate(df_list):
    d.name = df_name[i]

# Table 편집
for df in df_list:
    name = df.name[3:]
    new_col = df.iloc[0].tolist()[1:]
    new_index = pd.Series(df.iloc[2:, 0].tolist(), name='Order ID')
    value = df.iloc[2:,1:].to_numpy()
    df = pd.DataFrame(data = value, index = new_index, columns = new_col)

    df['Ship Mode'] = np.where(df['First Class'].isna() == False, 'First Class', np.where(df['Same Day'].isna() == False, 'Same Day', 
    np.where(df['Second Class'].isna() == False, 'Second Class', np.where(df['Standard Class'].isna() == False, 'Standard Class', None))))
    df['Sales'] = np.where(df['First Class'].isna() == False, df['First Class'], np.where(df['Same Day'].isna() == False, df['Same Day'], 
    np.where(df['Second Class'].isna() == False, df['Second Class'], np.where(df['Standard Class'].isna() == False, df['Standard Class'], None)))) 
    df['Segment'] = name

    df.drop(columns = df.columns[0:4], inplace=True)
    df = df[df['Ship Mode'].isna() == False]
    final_frame = pd.concat([final_frame, df])

In [7]:
final_frame = final_frame.reset_index().set_index('Segment')[['Ship Mode', 'Order ID', 'Sales']]

In [8]:
final_frame

,Ship Mode,Order ID,Sales
Segment,,,
consumer,Second Class,CA-2011-100706,129.44
consumer,Standard Class,CA-2011-100895,605.47
consumer,Second Class,CA-2011-101266,13.36
consumer,First Class,CA-2011-103366,149.95
consumer,Standard Class,CA-2011-104283,616.14
...,...,...,...
homeoffice,Standard Class,US-2014-132675,148.16
homeoffice,Second Class,US-2014-148768,71.976
homeoffice,Standard Class,US-2014-156083,9.664


In [9]:
final_frame.to_csv('Situtation1Solution1.csv', float_format="%g")

In [10]:
final_frame['Sales'] = final_frame['Sales'].astype('float')

In [11]:
#multi level indexing 활용

col_1 = ['Consumer', 'Corporate','Home Office']
col_2 = ['First Class', 'Same Day', 'Second Class', 'Standard Class']
cols = pd.MultiIndex.from_product([col_1, col_2], names=['Segment', 'Ship Mode'])
index = pd.Series(data.iloc[2:, 0].tolist(), name='Order ID')

value = data.iloc[2:, 1:].copy().to_numpy()

df = pd.DataFrame(data=value, columns=cols, index=index)


In [12]:
pivot_table = pd.DataFrame(df.unstack().dropna())
pivot_table.columns = ['Sales']
pivot_table = pivot_table.reset_index().set_index('Segment')

In [13]:
pivot_table

,Ship Mode,Order ID,Sales
Segment,,,
Consumer,First Class,CA-2011-103366,149.95
Consumer,First Class,CA-2011-109043,243.6
Consumer,First Class,CA-2011-113166,9.568
Consumer,First Class,CA-2011-124023,8.96
Consumer,First Class,CA-2011-130155,34.2
...,...,...,...
Home Office,Standard Class,US-2014-129224,4.608
Home Office,Standard Class,US-2014-132031,513.496
Home Office,Standard Class,US-2014-132297,598.31


In [16]:
pivot_table['Sales'] = pivot_table['Sales'].astype('float')

In [17]:
pivot_table.to_csv('Situtation1Solution2.csv', float_format='%g')